# CyJS Assembler

* Assemble a network from text using existing INDRA functionality
* Translate this into a model supported by cytoscape.js
* Visualize standalone, or within notebook
---

### I. Describe your network in plain English

* We have described the interactions in the McCormick RAS pathway map using plain English.
* Let's load this set of sentences.

In [1]:
from __future__ import absolute_import, print_function, unicode_literals
from builtins import dict, str
statement_path = 'ras_pathway.txt'
txt = open(statement_path, 'rt').read()
print(txt)

Growth-factor proteins activate EGFR, ERBB2 and FGFR.
Growth-factor proteins activate PDGFR.
Growth-factor proteins activate MET, ROS1 and ALK.
EGFR, ERBB2, PDGFR, MET, ROS1, ALK and FGFR activate GRB2 and SHC.
GRB2 and SHC activate RASGRF and SOS.
GRB2 binds SHC.
SOS and RASGRF activate HRAS, NRAS and KRAS.
RASGRP activates HRAS, KRAS and NRAS.
SPRY deactivates HRAS, KRAS and NRAS.
RASA/ARHGAP35 deactivates HRAS, NRAS and KRAS.
RASAL deactivates HRAS, NRAS and KRAS.
NF1/SPRED deactivates HRAS, NRAS and KRAS.
RASA/ARHGAP35 deactivates RHOA, RHOB and RHOC.
RASAL deactivates RHOA, RHOB and RHOC.
NF1/SPRED deactivates RHOA, RHOB and RHOC.
HRAS, NRAS and KRAS activate RALGDS.
RALGDS activates RALA and RALB.
HRAS, NRAS and KRAS activate ARAF, BRAF and RAF1.
ARAF, BRAF and RAF1 activate MAP2K1 and MAP2K2.
MAP2K1 and MAP2K2 activate MAPK1 and MAPK3.
MAPK1 and MAPK3 activate ETS, JUN and FOS.
KSR binds ARAF, BRAF and RAF1.
KSR binds MAP2K1 and MAP2K2.
KSR binds MAPK1 and MAPK3.
ETS, FOS and JU

---
### II. Process text into INDRA statements using REACH parser

* If we have never parsed this text before (or we've made changes to it) we should process it with REACH
* This will yield a set of INDRA statements
* We will then pickle the statements so we don't have to parse them every time we run this script.
    * Change the cell below to from Code -> Raw NBConvert in order to not execute it every time

In [2]:
from indra.sources import reach
rp = reach.process_text(txt, offline=False)
st = rp.statements

In [3]:
from indra.preassembler.grounding_mapper import GroundingMapper, gm

GM = GroundingMapper(gm)
mapped_stmts = GM.map_agents(st, do_rename=True)

import pickle
pickle.dump(mapped_stmts, open("st.pkl", "wb"))

---
### III. Assemble an INDRA model

In [4]:
import pickle
st = pickle.load(open("st.pkl", "rb"))

#### What do the statements look like?
* Let's see.

In [5]:
st

[Complex(GRB2(), SHC()),
 Complex(KSR(), RAF1()),
 Complex(KSR(), BRAF()),
 Complex(KSR(), ARAF()),
 Complex(KSR(), MAP2K1()),
 Complex(KSR(), MAP2K2()),
 Complex(KSR(), MAPK3()),
 Complex(KSR(), MAPK1()),
 Activation(Growth_factor_proteins(), activity, ERBB2(), activity, True),
 Activation(Growth_factor_proteins(), activity, FGFR(), activity, True),
 Activation(Growth_factor_proteins(), activity, EGFR(), activity, True),
 Activation(Growth_factor_proteins(), activity, PDGFR(), activity, True),
 Activation(Growth_factor_proteins(), activity, ALK(), activity, True),
 Activation(Growth_factor_proteins(), activity, ROS1(), activity, True),
 Activation(PDGFR(), activity, GRB2(), activity, True),
 Activation(ERBB2(), activity, GRB2(), activity, True),
 Activation(ALK(), activity, SHC(), activity, True),
 Activation(PDGFR(), activity, SHC(), activity, True),
 Activation(FGFR(), activity, GRB2(), activity, True),
 Activation(ALK(), activity, GRB2(), activity, True),
 Activation(ROS1(), activi

#### Import some necessities
* these let us work with INDRA statements and the CyJSAssembler

In [6]:
from indra.statements import *
from indra.assemblers import CyJSAssembler

* **Instntiate a CyJSAssembler**

In [7]:
cja = CyJSAssembler()

* **add the statements into the assembler**

In [8]:
cja.add_statements(st)

### make_model
* **grouping** will combine structurally identical nodes under a group node
* **virtual edges** are the edges that belong to grouped nodes. They are not visualized but should be maintained for our default layout
* **add_edge_weights** weighs the edges of nodes depending on how many members they have. Again, this is used in our default layout.

In [9]:
cja.make_model(grouping=True, 
               drop_virtual_edges=False,
               add_edge_weights=True)

'{\n "mut_colorscale_str": "[]", \n "model_elements_str": "{\\n \\"edges\\": [\\n  {\\n   \\"data\\": {\\n    \\"i\\": \\"Virtual\\", \\n    \\"id\\": 0, \\n    \\"polarity\\": \\"negative\\", \\n    \\"source\\": 1, \\n    \\"target\\": 2, \\n    \\"weight\\": 1\\n   }\\n  }, \\n  {\\n   \\"data\\": {\\n    \\"i\\": \\"Virtual\\", \\n    \\"id\\": 3, \\n    \\"polarity\\": \\"negative\\", \\n    \\"source\\": 1, \\n    \\"target\\": 4, \\n    \\"weight\\": 1\\n   }\\n  }, \\n  {\\n   \\"data\\": {\\n    \\"i\\": \\"Virtual\\", \\n    \\"id\\": 5, \\n    \\"polarity\\": \\"positive\\", \\n    \\"source\\": 6, \\n    \\"target\\": 7, \\n    \\"weight\\": 1\\n   }\\n  }, \\n  {\\n   \\"data\\": {\\n    \\"i\\": \\"Virtual\\", \\n    \\"id\\": 8, \\n    \\"polarity\\": \\"positive\\", \\n    \\"source\\": 6, \\n    \\"target\\": 9, \\n    \\"weight\\": 1\\n   }\\n  }, \\n  {\\n   \\"data\\": {\\n    \\"i\\": \\"Virtual\\", \\n    \\"id\\": 10, \\n    \\"polarity\\": \\"negative\\", \\n   

### set_context
* We integrate gene expression and mutation status into our visualization
    * Gene expression is log10 transformed and displayed by color intensity on an **n_bins** scale
    * Mutational status is represented by green (wt) and orange (mutated)
* Here, we chose the LOXIMVI_SKIN melanoma cell line from CCLE

In [10]:
cja.set_context(cell_type = 'LOXIMVI_SKIN', 
                bin_expression = True, 
                n_bins = 9)

### save_model
* We save the model as 'model.js' which is the file expected by our visualizer
* must be saved in cyjs folder where the html and js files for the visualization reside

In [11]:
cja.save_model('cyjs/model.js')

---
# Display the assembled model

In [13]:
%%html
<iframe width="100%" height="600" src="cyjs/index.html"></iframe>